In [1]:
#Extracting entity-adjective pairs from news articles per news media source (outlets)
#It requires NE cluster pickle files generated by clustering named entities and 
#stores the extracted entity-adj pairs in a pickle (json format)

import pandas as pd
import pickle
import sys
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import timeit
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import operator

from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
import sklearn

from pprint import pprint

nlp = spacy.load('en_core_web_lg')
stop_words = set(stopwords.words('english')) 

In [2]:
#data = pd.read_csv('../News_Media_bias/Git Code/Dataset/articles1.csv')
data = pd.read_csv('coref_resolved_news_article_dataset.csv')

In [3]:
def save_pickle_data(filepath, final_ent_adj_pairs):
    with open(filepath, 'wb') as f:
        pickle.dump(final_ent_adj_pairs, f)

def get_entity_counts_and_pos_tags(text, col):

    entity_count = dict()
    entity_adj_pairs = dict()
    doc = nlp(text)
    ner = {}
    pos_tags_adj = [(i, i.tag_) for i in doc if i.tag_ == 'JJ']
    for ent in doc.ents: 
        if ent.label_ in ner:
            ner[ent.label_].append(ent.text)
        else:
            ner[ent.label_] = [ent.text]
        
    if(col in ner):
        entities = ner[col]
        for ent in entities:
            if ent in entity_count:           
                entity_count[ent]+=1
            else:
                entity_count[ent] = 1
                
            for adj in pos_tags_adj:
                if(ent in entity_adj_pairs):
                    if(adj in entity_adj_pairs[ent]):
                        entity_adj_pairs[ent][adj[0]] += 1
                    else:
                        entity_adj_pairs[ent][adj[0]] = 1
                else:
                    entity_adj_pairs[ent] = {adj[0]:1}
   
    return entity_count, entity_adj_pairs
    

def get_entity_adjective_pairs(news_article):
    
    ent_count_per_art, ent_adj_pair_per_art = dict(), dict()
    token_sent = sent_tokenize(news_article)
    for sent in token_sent:
        entity_count, entity_adj_pairs= get_entity_counts_and_pos_tags(sent, 'PERSON')
        for ent in entity_count:
            if ent in ent_count_per_art:
                ent_count_per_art[ent] += entity_count[ent]
            else:
                ent_count_per_art[ent] = entity_count[ent]

        for ent in entity_adj_pairs:
            if ent not in ent_adj_pair_per_art:
                ent_adj_pair_per_art[ent] = dict()
            for adj in entity_adj_pairs[ent]:
                adj = str(adj).strip()
                if(adj in ent_adj_pair_per_art[ent]):
                    ent_adj_pair_per_art[ent][adj] += 1
                else:
                    ent_adj_pair_per_art[ent][adj] = 1
                    
    return ent_count_per_art, ent_adj_pair_per_art   


def get_entity_adj_pairs(news_article_list):
    
    entity_adj_pairs = dict()
    count = 1
    for news_article in news_article_list:
        print(count, end=" ")
        count+=1
        ent_count_per_arts, ent_adj_pair_per_art = get_entity_adjective_pairs(news_article)

        for ent in ent_adj_pair_per_art:
            if(ent not in entity_adj_pairs):
                entity_adj_pairs[ent] = {'ent_count':ent_count_per_arts[ent], 'adjectives':{}}

            for adj in ent_adj_pair_per_art[ent]:
                adj = str(adj).strip()
                if adj in entity_adj_pairs[ent]['adjectives']:
                    entity_adj_pairs[ent]['adjectives'][adj] += ent_adj_pair_per_art[ent][adj]
                else:
                    entity_adj_pairs[ent]['adjectives'][adj] = ent_adj_pair_per_art[ent][adj]
                    
    return entity_adj_pairs


def get_pickle_data(filepath):
    ent_clusters = None
    with open(filepath, 'rb') as f:
        ent_clusters = pickle.load(f)
    return ent_clusters

def get_entity_counts(entity_adj_pairs):
    
    keys = list(entity_adj_pairs.keys())
    ent_counts = dict()
    for k in keys:
        ent_counts[k] = entity_adj_pairs[k]['ent_count']
    return ent_counts
    
def get_most_frequent_entities(entity_adj_pairs, ent_counts, num_of_ent):
    
    sorted_ent_count = sorted(ent_counts.items(), key=operator.itemgetter(1))
    top_ent = sorted_ent_count[-num_of_ent:]
    top_entity_adj_pairs = dict()
    for ent, count in top_ent:
        top_entity_adj_pairs[ent] = entity_adj_pairs[ent]
    return top_entity_adj_pairs
    
def get_entity_cluster_map(entity_adj_pairs, ent_clusters):
    
    ent_cls_map = dict()
    for ent in entity_adj_pairs:
        for key in ent_clusters:
            cls = ent_clusters[key]
            if(ent in cls):
                ent_cls_map[ent] = key
    return ent_cls_map

def get_cluster_ent_map(ent_cls_map):

    cls_ent_map = dict()
    for key in ent_cls_map:
        cls_id = ent_cls_map[key]
        if(cls_id in cls_ent_map):
            cls_ent_map[cls_id].append(key)
        else:
            cls_ent_map[cls_id] = [key]
            
    return cls_ent_map

def get_final_ent_adj_pairs(entity_adj_pairs, ent_cls_map):
    
    final_ent_adj_pairs = dict()
    count = 0
    l = len(entity_adj_pairs)
    for ent in entity_adj_pairs:
        count+=1
        if ent in ent_cls_map:
            cls_id = ent_cls_map[ent]
            if(cls_id not in final_ent_adj_pairs):
                final_ent_adj_pairs[cls_id] = {'ent_count':entity_adj_pairs[ent]['ent_count'], 'adjectives':entity_adj_pairs[ent]['adjectives']}

            else:
                final_ent_adj_pairs[cls_id]['ent_count'] += entity_adj_pairs[ent]['ent_count']
                for adj in entity_adj_pairs[ent]['adjectives']:
                    adj = str(adj).strip()
                    if adj in final_ent_adj_pairs[cls_id]['adjectives']:
                        final_ent_adj_pairs[cls_id]['adjectives'][adj] += entity_adj_pairs[ent]['adjectives'][adj]
                    else:
                        final_ent_adj_pairs[cls_id]['adjectives'][adj] = entity_adj_pairs[ent]['adjectives'][adj]

    return final_ent_adj_pairs


def add_entity_cluster_information(final_ent_adj_pairs, cls_ent_map):
    
    for key in final_ent_adj_pairs:
        cls = cls_ent_map[key]
        final_ent_adj_pairs[key]['person_entities'] = cls
        
    return final_ent_adj_pairs

In [4]:
c = data['publication']
print(set(c))

{'New York Times', 'CNN', 'Atlantic', ' â€\x9d Corkin said ', 'Breitbart', ' than Clinton 45 ', 'Business Insider'}


In [5]:
def get_news_data(data, media, number_of_news_articles):
    
    filter1 = data['publication'] == media
    news = data.loc[filter1]
    index = np.random.choice(len(news), number_of_news_articles, replace=False)
    news = news.sample(n = number_of_news_articles) 
    news = [(row['title'],row['content'], row['publication']) for index, row in news.iterrows()]
    
    return news

publishers = ['Breitbart', 'New York Times', 'Atlantic', 'CNN', 'Business Insider']
data_by_publisher = get_news_data(data, publishers[4], 5000)
news_article_list = [d[1] for d in data_by_publisher]
#print(data_by_publisher[0][1], len(data_by_publisher))
#print(len(news_article_list), news_article_list[0])


In [6]:
filepath = "Nammed_entities/clusters_ne_.pickle"
ent_clusters = get_pickle_data(filepath)

entity_adj_pairs = get_entity_adj_pairs(news_article_list)
ent_counts = get_entity_counts(entity_adj_pairs)
top_entity_adj_pairs = get_most_frequent_entities(entity_adj_pairs, ent_counts, 10000)

# filepath = "Nammed_entities/top_10000_entity_adj_pairs.pickle"
# save_pickle_data(filepath, top_entity_adj_pairs)

ent_cls_map = get_entity_cluster_map(entity_adj_pairs, ent_clusters)
cls_ent_map = get_cluster_ent_map(ent_cls_map)

final_ent_adj_pairs = get_final_ent_adj_pairs(entity_adj_pairs, ent_cls_map)
final_ent_adj_pairs = add_entity_cluster_information(final_ent_adj_pairs, cls_ent_map)
        
# filepath = "Nammed_entities/final_clusters_ne_2.pickle"
# save_pickle_data(filepath, final_ent_adj_pairs)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

1861 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 

3500 3501 3502 3503 3504 3505 3506 3507 3508 3509 3510 3511 3512 3513 3514 3515 3516 3517 3518 3519 3520 3521 3522 3523 3524 3525 3526 3527 3528 3529 3530 3531 3532 3533 3534 3535 3536 3537 3538 3539 3540 3541 3542 3543 3544 3545 3546 3547 3548 3549 3550 3551 3552 3553 3554 3555 3556 3557 3558 3559 3560 3561 3562 3563 3564 3565 3566 3567 3568 3569 3570 3571 3572 3573 3574 3575 3576 3577 3578 3579 3580 3581 3582 3583 3584 3585 3586 3587 3588 3589 3590 3591 3592 3593 3594 3595 3596 3597 3598 3599 3600 3601 3602 3603 3604 3605 3606 3607 3608 3609 3610 3611 3612 3613 3614 3615 3616 3617 3618 3619 3620 3621 3622 3623 3624 3625 3626 3627 3628 3629 3630 3631 3632 3633 3634 3635 3636 3637 3638 3639 3640 3641 3642 3643 3644 3645 3646 3647 3648 3649 3650 3651 3652 3653 3654 3655 3656 3657 3658 3659 3660 3661 3662 3663 3664 3665 3666 3667 3668 3669 3670 3671 3672 3673 3674 3675 3676 3677 3678 3679 3680 3681 3682 3683 3684 3685 3686 3687 3688 3689 3690 3691 3692 3693 3694 3695 3696 3697 3698 3699 

In [7]:
pprint(final_ent_adj_pairs)
filepath = "Nammed_entities/ent_adj_pairs_BI.pickle"
save_pickle_data(filepath, final_ent_adj_pairs)

{6: {'adjectives': {'embarrassing': 1, 'past': 1, 'potential': 1, 'real': 1},
     'ent_count': 1,
     'person_entities': ['Howard Stern']},
 9: {'adjectives': {'Jewish': 1, 'old': 1, 'political': 1},
     'ent_count': 1,
     'person_entities': ['Jonathan Greenblatt']},
 10: {'adjectives': {'former': 1},
      'ent_count': 1,
      'person_entities': ['Candace Smith']},
 12: {'adjectives': {'Remain': 2,
                     'big': 1,
                     'final': 1,
                     'financial': 1,
                     'likely': 1,
                     'local': 1,
                     'much': 4,
                     'narrow': 1,
                     'official': 1,
                     'prominent': 1,
                     'second': 1,
                     '™': 2},
      'ent_count': 4,
      'person_entities': ['Nigel Farage']},
 16: {'adjectives': {'former': 1, 'proxy': 1},
      'ent_count': 2,
      'person_entities': ['Eric Garcetti']},
 17: {'adjectives': {'American': 1,
    

                      'Olympic': 1,
                      'big': 1,
                      'classified': 5,
                      'covert': 1,
                      'factual': 1,
                      'first': 4,
                      'hostile': 1,
                      'key': 3,
                      'last': 4,
                      'other': 1,
                      'particular': 1,
                      'past': 2,
                      'personal': 1,
                      'possible': 1,
                      'presidential': 1,
                      'private': 6,
                      'reasonable': 1,
                      'second': 1,
                      'unclassified': 1,
                      '™': 1},
       'ent_count': 2,
       'person_entities': ['Matt Lauer']},
 173: {'adjectives': {'clear': 1,
                      'direct': 1,
                      'former': 2,
                      'inappropriate': 1,
                      'transgressive': 1,
                      '™': 1},

                      'general': 1,
                      'good': 1,
                      'gubernatorial': 2,
                      'inaccurate': 1,
                      'incoming': 11,
                      'last': 1,
                      'legal': 1,
                      'legislative': 2,
                      'liberal': 3,
                      'local': 2,
                      'multiple': 1,
                      'new': 2,
                      'other': 1,
                      'outgoing': 2,
                      'own': 2,
                      'political': 2,
                      'potential': 1,
                      'presidential': 1,
                      'private': 2,
                      'proper': 1,
                      'responsible': 1,
                      'several': 1,
                      'special': 1,
                      'statewide': 1,
                      'substantial': 1,
                      'typographic': 1,
                      'unconstitutional': 3,


                      'sick': 1,
                      'significant': 5,
                      'silly': 1,
                      'similar': 3,
                      'single': 1,
                      'slim': 1,
                      'small': 2,
                      'smart': 1,
                      'social': 1,
                      'soft': 1,
                      'sorry': 1,
                      'southern': 2,
                      'special': 2,
                      'specific': 2,
                      'steep': 1,
                      'straightforward': 1,
                      'strong': 4,
                      'stunning': 1,
                      'successful': 1,
                      'such': 3,
                      'tear': 1,
                      'tense': 1,
                      'terrible': 1,
                      'terrific': 5,
                      'terrorist': 1,
                      'testy': 1,
                      'then': 1,
                      'third': 5,
       

 655: {'adjectives': {'dedicated': 1, 'main': 1, 'social': 1},
       'ent_count': 1,
       'person_entities': ['Hailey Baldwin']},
 673: {'adjectives': {'chief': 1, 'cyber': 1, 'sophisticated': 1},
       'ent_count': 1,
       'person_entities': ['Chris Wysopal']},
 677: {'adjectives': {'old': 6},
       'ent_count': 6,
       'person_entities': ['Paul Dahlen']},
 682: {'adjectives': {'Brazilian': 23,
                      'Congressional': 2,
                      'bitter': 1,
                      'budgetary': 2,
                      'central': 1,
                      'certain': 1,
                      'dark': 1,
                      'decisive': 1,
                      'disproportionate': 1,
                      'economic': 4,
                      'female': 1,
                      'few': 1,
                      'first': 1,
                      'fiscal': 1,
                      'full': 1,
                      'furious': 1,
                      'future': 1,
             

...

 2693: {'adjectives': {'Former': 2,
                       'Prominent': 2,
                       'Republican': 4,
                       'additional': 1,
                       'aggressive': 1,
                       'anxious': 1,
                       'ardent': 1,
                       'black': 3,
                       'boisterous': 1,
                       'congressional': 1,
                       'devoted': 5,
                       'dynamic': 1,
                       'early': 1,
                       'elite': 5,
                       'enthusiastic': 1,
                       'few': 2,
                       'final': 1,
                       'formal': 1,
                       'former': 29,
                       'hopeful': 1,
                       'imminent': 1,
                       'independent': 1,
                       'intense': 1,
                       'last': 2,
                       'little': 1,
                       'long': 1,
                       'many':

                       'recreational': 2,
                       'red': 6,
                       'regular': 3,
                       'related': 1,
                       'reliable': 2,
                       'reluctant': 1,
                       'remarkable': 1,
                       'reputable': 1,
                       'respective': 5,
                       'responsible': 4,
                       'rich': 4,
                       'right': 3,
                       'rigid': 1,
                       'rmont': 1,
                       'rocky': 1,
                       'running': 2,
                       'safe': 6,
                       'same': 28,
                       'satisfied': 1,
                       'scared': 1,
                       'scary': 1,
                       'scientific': 1,
                       'second': 7,
                       'secret': 4,
                       'secretive': 2,
                       'senior': 24,
                       'sensational'

                       'unpopular': 1,
                       '™': 10},
        'ent_count': 2,
        'person_entities': ['Bob Corker']},
 2817: {'adjectives': {'Dem': 1,
                       'absent': 1,
                       'clear': 1,
                       'former': 1,
                       'senior': 1},
        'ent_count': 1,
        'person_entities': ['Dan Pfeiffer']},
 2823: {'adjectives': {'British': 6,
                       'Chinese': 1,
                       'absent': 8,
                       'full': 1,
                       'immediate': 1,
                       'independent': 1,
                       'keen': 1,
                       'other': 1,
                       'present': 1,
                       'several': 1,
                       'typical': 1,
                       'various': 1,
                       'vulnerable': 1},
        'ent_count': 1,
        'person_entities': ['George Osborne']},
 2829: {'adjectives': {'bad': 1,
                       'br

                       'Diplomatic': 1,
                       'Mexican': 1,
                       'Russian': 1,
                       'commemorative': 1,
                       'contentious': 1,
                       'national': 1,
                       'next': 1,
                       'recent': 1,
                       'tense': 1,
                       '™': 4},
        'ent_count': 4,
        'person_entities': ['Malcolm Turnbull']},
 3002: {'adjectives': {'2nd': 1,
                       'Libertarian': 2,
                       'Renowned': 6,
                       'Republican': 8,
                       'Statistical': 2,
                       'accurate': 1,
                       'chief': 1,
                       'cultural': 1,
                       'electable': 1,
                       'final': 1,
                       'fresh': 1,
                       'general': 1,
                       'improbable': 1,
                       'institutional': 1,
                    

                       'lethal': 1,
                       'lewd': 1,
                       'likely': 1,
                       'literary': 1,
                       'local': 1,
                       'longtime': 1,
                       'malware': 1,
                       'many': 1,
                       'maximum': 2,
                       'middle': 1,
                       'negative': 1,
                       'new': 4,
                       'numerous': 1,
                       'open': 1,
                       'other': 4,
                       'overblown': 1,
                       'own': 8,
                       'personal': 9,
                       'philanthropic': 1,
                       'political': 3,
                       'possible': 1,
                       'potential': 1,
                       'presidential': 15,
                       'primary': 1,
                       'private': 11,
                       'prolonged': 1,
                       'public': 3,

In [8]:
ent_adj = get_pickle_data(filepath)
print(len(final_ent_adj_pairs), len(ent_adj))

769 769
